Questions:
1. Establish G&S per time period, and try to find the latent variables that influence discounts?
2. How often do UW values significantly differ from actual?
3. Are we able to estimate repair reserve amounts?

# Imports

In [1]:
import re, random
import pandas as pd
import numpy  as np
import plotly.graph_objects as go
from ipywidgets import widgets

In [2]:
fnma     = pd.read_csv('~/tapcap/data/data_notebooks/fnma_6_4.csv')
counties = pd.read_csv('~/tapcap/data/data_notebooks/county_markets.csv')

/Users/zr/miniconda3/envs/tc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,70,71,90,91,125,146,150,151,152,160,161,162,171,185,189,190,196,202,208,214,217,220,222,223,224,228,229,230,234,235,236,240,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Colors used for graph viz
# Thank you https://materialuicolors.co/ & https://flatuicolors.com/
colors = {
    'red'            : 'rgba(244, 67, 54,1.0)',
    'blue'           : 'rgba(33, 150, 243,1.0)',
    'light_green'    : 'rgba(139, 195, 74,1.0)',
    'deep_orange'    : 'rgba(255, 87, 34,1.0)',
    'pink'           : 'rgba(233, 30, 99,1.0)',
    'light_blue'     : 'rgba(3, 169, 244,1.0)',
    'lime'           : 'rgba(205, 220, 57,1.0)',
    'brown'          : 'rgba(121, 85, 72,1.0)',
    'purple'         : 'rgba(156, 39, 176,1.0)',
    'nother_orange'  : 'rgba(243, 156, 18,1.0)',
    'cyan'           : 'rgba(0, 188, 212,1.0)',
    'yellow'         : 'rgba(255, 235, 59,1.0)',
    'grey'           : 'rgba(158, 158, 158,1.0)',
    'pomegranate'    : 'rgba(192, 57, 43,1.0)',
    'deep_purple'    : 'rgba(103, 58, 183,1.0)',
    'teal'           : 'rgba(0, 150, 136,1.0)',
    'amber'          : 'rgba(255, 193, 7,1.0)',
    'blue_grey'      : 'rgba(96, 125, 139,1.0)',
    'indigo'         : 'rgba(63, 81, 181,1.0)',
    'green'          : 'rgba(76, 175, 80,1.0)',
    'orange'         : 'rgba(255, 152, 0,1.0)',
    'emerald'        : 'rgba(46, 204, 113,1.0)',
    'sun_flower'     : 'rgba(241, 196, 15,1.0)',
    'pumpkin'        : 'rgba(211, 84, 0,1.0)',
    'amethyst'       : 'rgba(155, 89, 182,1.0)',
    'midnight_blue'  : 'rgba(44, 62, 80,1.0)',
    'carrot'         : 'rgba(230, 126, 34,1.0)',
    'jackson_purple' : 'rgba(64, 64, 122,1.0)',
    'mandarin_sorbet': 'rgba(255, 177, 66,1.0)',
    'butternut'      : 'rgba(255, 218, 121,1.0)',
    'liberty'        : 'rgba(71, 71, 135,1.0)',
    'celestial_green': 'rgba(51, 217, 178,1.0)',
    'deep_red'       : 'rgba(179, 57, 57,1.0)'
}

def create_fill_rgba(rgba):
    return rgba[:-4] + '0.2)'

palette  = {color: {'primary': rgba, 'fill': create_fill_rgba(rgba)} for color, rgba in colors.items()}
no_color = 'rgba(255,255,255,0)'

# Clean

In [4]:
# Create a deep copy for use if we screw up the original dataframe
try: # If copy already exists
    fnma = fnma.copy()
except: # create the copy
    fnma = fnma.copy()

## Columns

In [5]:
# Drop unnecessary cols
security_cols_to_remove = [
    'Security Prefix', 'Security Prefix',  'Security Type',  'Security ARM Subtype', 'Security Status',  
    'Security UPB ($) - Current', 'Security Minimum Loan Issuance UPB ($)', 'Security Maximum Loan Issuance UPB ($)', 'Security WA Issuance UPB ($)',  
    'Security Number of Loans - Current', 'Security Number of Loans - Issuance', 'Security Number of Properties - Current', 
    'Security Number of Properties - Issuance', 'Security Resecuritization', 'Security Settlement Date',
    'Security First Payment Date', 'Security First Scheduled Payment Change Date in Trust', 'Security Next Rate Change Date', 
    'Security Next Distribution Date', 'Security Resecuritized Date', 'Security Terminated Date', 'Security Dissolved Date', 'Security Maturity Date', 
    'Security Factor', 'Security Paying PTR (%) - Current', 'Security Paying PTR (%) - Issuance','Security Minimum Loan PTR (%)','Security Maximum Loan PTR (%)',
    'Security WA Accruing Note Rate (%) - Current ','Security WA Accruing Note Rate (%) - Issuance','Security WA Note Rate Cap (%) - Current',
    'Security WA Note Rate Floor (%) - Current','Security WA Original Term - Current', 'Security WA Original Term - Issuance','Security WA Remaining Term - Current',
    'Security WA Remaining Term - Issuance', 'Security WA Seasoning - Current', 'Security WA Seasoning - Issuance', 'Security WA Amortization Term',
    'Security WA Original I/O Term', 'Security WA MBS Margin (%) - Current', 'Security WA Loan Margin Rate (%) - Issuance', 'Security WA Loan Margin Rate (%) - Current',
    'Security WA Months to Rate Change - Current','Security WA Months to Rate Change - Issuance','Security WA Accruing PTR (%) - Current','Security WA Lifetime PTR Cap (%) - Current',
    'Security WA Lifetime PTR Floor (%) - Current','Security WA LTV (%) - Issuance ','Security WA NCF DSCR (x) - Current','Security WA NCF DSCR (x) - Issuance',
    'Security ARM Index', 'Security Convertible', 'Security Lookback Period', 'Security Rate Reset Frequency', 'Security Payment Reset Frequency', 
    'Security Periodic Rate Change (%)', 'Security Periodic Payment Change (%)', 'Security Prepayment Type', 'Security WA Prepayment Term', 
    'Security Prepayment Latest End Date', 'Security Number of Prepaid Loans', 'Security Prepaid Loans Principal Amount ($)', 'Security Prepayment Allocation ($)',
    'Security Cumulative Prepayment Allocation ($)', 'Security Total Interest Distribution ($)', 'Security Total Principal Distribution ($)'
]

loan_cols_to_remove = [
    'Loan Defeasance Status',
    'Loan Payment Status', 'Loan UPB - Current ($)', 'Loan Total Debt Current Balance ($) - Issuance','Loan Balance per Unit ($)','Loan Tier Drop Eligible',
    'Loan Cross-Collateralized Loan', 'Loan Cross Defaulted Loan', 'Loan First Scheduled Rate Change Date in Trust', 'Loan First Scheduled Payment Change Date in Trust',
    'Loan Next Rate Change Date', 'Loan Next Payment Change Date', 'Loan ARM Index',
    'Loan ARM Margin (%) ', 'Loan Note Rate Cap (%) ', 'Loan Note Rate Floor (%) ', 'Loan Periodic Rate Change Increase (%) ','Loan Periodic Rate Change Decrease (%) ',
    'Loan Periodic Payment Change Increase (%) ', 'Loan Periodic Payment Change Decrease (%) ', 'Loan Rate Reset Frequency', 'Loan Payment Reset Frequency',
    'Loan Lookback Period', 'Loan Convertible', 'Loan Conversion Eligibility Start Date', 'Loan Conversion Eligibility End Date', 'Loan Initial Fixed Term',
    'Loan Initial Fixed Term End Date', 'Loan Fixed Principal Payment Amount ($)', 'Loan Accrual Method', 'Loan Remaining Term', 'Loan Seasoning - Current',
    'Loan Seasoning - Issuance', 'Loan UW NCF DSCR (I/O) (x)', 'Loan UW NCF DSCR at Cap (x)', 'Loan All-In Current NCF DSCR (x)', 'Loan All-In UW NCF DSCR (x)',
    'Loan Current NCF Debt Yield (%)', 'Loan UW NCF Debt Yield (%)', 'Loan Issuance DSCR (x) (Pre-DUS Disclose)', 'Loan DSCR at Maximum Payment  (x) (Pre-DUS Disclose)',
    'Loan All-In At-Issuance LTV (%)', 'Loan Monthly Debt Service Amount ($) - Partial IO', 'Loan Scheduled Interest Amount ($) ', 'Loan Scheduled Principal Amount ($) ',
    'Loan Total Scheduled P&I Due ($) ', 'Loan Liquidation / Prepayment Code', 'Loan Liquidation / Prepayment Date', 'Loan Prepayment Premium / Yield Maintenance (YM) Received ($) ',
    'Loan Additional Debt Type', 'Loan Additional Debt Pool Number & Loan Number', 'Loan Additional Debt Balance ($) ', 'Loan Additional Debt Monthly Payment ($)',
    'Loan Additional Debt Maximum Monthly Payment ($)', 'Loan Additional Debt Lien Priority', 'Loan Additional Debt Maturity Date', 'Loan Mezzanine Loan Balance ($) ',
    'Loan Mezzanine Monthly Payment ($)', 'Loan Mezzanine Loan Initial Term Maturity Date'
]

property_cols_to_remove = [
    'Property Status', 'Property Ownership Interest', 'Property HAP Remaining Term', 'Property Tax Escrow', 'Property Terrorism Insurance', 
    'Property  Condition Rating As of Date', 'Property Green Performance Preceding Year', 
    'Property Green Performance Preceding Year ENERGY STAR¬Æ Score', 'Property Green Performance Preceding Year EPA Water Score', 
    'Property Green Performance Preceding Year Source Energy Use Intensity', 'Property Green Performance Preceding Year Water Use Intensity', 
    'Property Green Performance Preceding Year Energy Score Date', 'Property Green Performance 2nd Preceding Year', 
    'Property Green Performance 2nd Preceding Year ENERGY STAR¬Æ Score', 'Property Green Performance 2nd Preceding Year EPA Water Score', 
    'Property Green Performance 2nd Preceding Year Source Energy Use Intensity', 'Property Green Performance 2nd Preceding Year Water Use Intensity', 
    'Property Green Performance 2nd Preceding Year Energy Score Date', 'Property Green Performance 3rd Preceding Year', 
    'Property Green Performance 3rd Preceding Year ENERGY STAR¬Æ Score','Property Green Performance 3rd Preceding Year EPA Water Score', 'Property Green Performance 3rd Preceding Year Source Energy Use Intensity',
    'Property Green Performance 3rd Preceding Year Water Use Intensity', 'Property Green Performance 3rd Preceding Year Energy Score Date',
]

credit_facility_cols_to_remove = [
    'Credit Facility Deal Max Borrowing Amount ($)',
    'Credit Facility Deal Max Permitted LTV (%)', 'Credit Facility Deal Min Permitted DSCR (x) - Fixed', 'Credit Facility Deal Min Permitted DSCR (x) - ARM',
    'Credit Facility Deal Additional Loans Allowed', 'Credit Facility Deal Property Release Allowed', 'Credit Facility Deal Property Substitution Allowed',
    'Credit Facility Deal Property Additions Allowed', 'Credit Facility Deal Total UPB ($)', 'Credit Facility Deal Number of Properties', 'Credit Facility Deal LTV (%)',
    'Credit Facility Deal Property Value ($)', 'Credit Facility Deal Total Collateral Value ($)', 'Credit Facility Deal Other Collateral - Type', 
    'Credit Facility Deal Other Collateral -  Amount ($)', 'Credit Facility Deal UW NCF DSCR (x)', 'Credit Facility Deal UW NCF DSCR I/O (x)', 
    'Credit Facility Deal Preceding Year', 'Credit Facility Deal Preceding NCF DSCR (x)', 'Credit Facility Deal 2nd Preceding Year',
    'Credit Facility Deal 2nd Preceding NCF DSCR (x)', 'Credit Facility Deal 3rd Preceding Year', 'Credit Facility Deal 3rd Preceding NCF DSCR (x)'
]

other_cols_to_remove = [
    'Additional Disclosure Indicator'
]

fnma = fnma.drop(columns = security_cols_to_remove + loan_cols_to_remove + property_cols_to_remove + credit_facility_cols_to_remove + other_cols_to_remove)

In [6]:
# Standardize/Sanitize column names
def fix_col_names(col):
    col = col.strip()
    col = col.lower()
    col = col.replace('(x)', '')
    col = col.replace('($)', '')
    col = col.replace('-', '')
    col = col.replace('/', '')
    col = col.strip()
    col = col.replace('  ', ' ')
    col = col.replace(' ','_')
    col = col.replace('__', '_')
    col = col.replace('(%)', '%')
    
    return col

fnma.columns = [fix_col_names(col) for col in list(fnma.columns)]

## Values

In [7]:
# For this analysis, only consider fixed rate records
fnma = fnma[fnma['loan_interest_type']=='Fixed']

# Drop records with no loan amount (loan amount used in graph filters)
fnma = fnma.dropna(subset=['loan_original_upb'])

# Fix several categorical columns
fnma['property_affordable_housing_type'] = fnma['property_affordable_housing_type'].apply(lambda x: 'None' if pd.isnull(x) else x)
fnma['security_green']     = fnma['security_green'].apply(lambda x: 'N' if pd.isnull(x) else x)
fnma['loan_lien_position'] = fnma['loan_lien_position'].apply(lambda x: 'First' if pd.isnull(x) else x)
fnma['loan_purpose']       = fnma['loan_purpose'].apply(lambda x: 'Acquisition' if x=='Purchase' else x)

# Consolidate prepay values
def fix_prepay_type(prepay_str):
    if prepay_str == 'See Issuance Documents':
        return 'Other Prepayment'
    elif prepay_str == '1% Fixed Prepayment':
        return 'Other Prepayment'
    elif prepay_str == 'Defeasance':
        return 'Yield Maintenance'
    elif prepay_str == 'Prepayment Lockout':
        return 'Other Prepayment'
    elif pd.isnull(prepay_str):
        return 'Other Prepayment'

    return prepay_str
fnma['loan_prepayment_type'] = fnma['loan_prepayment_type'].apply(fix_prepay_type)

# Consolidate columns property_general_property_type & property_specific_property_type
def consolidate_property_type(row):
    if row['property_general_property_type'] == 'Manufactured Housing':
        return 'Manufactured Housing'
    if row['property_general_property_type'] == 'Cooperative Housing':
        return 'Cooperative'
    return row['property_specific_property_type']
fnma['property_type'] = fnma.apply(consolidate_property_type, axis=1)

def convert_currency_to_float(currency):
    currency = str(currency).replace(',','')
    currency = float(currency)
    return currency
fnma['loan_original_upb'] = fnma['loan_original_upb'].apply(convert_currency_to_float)

# Standardize Lender Names
lender_rename_obj = {
    'PNC Bank, National Association': 'PNC',
    'Centerline Mortgage Capital Inc.': 'Centerline',
    'Berkadia Commercial Mortgage LLC': 'Berkadia',
    'Walker & Dunlop, LLC': 'Walker',
    'Wells Fargo Bank, N.A.': 'Wells',
    'Prudential Multifamily Mortgage, LLC': 'Prudential',
    'Enterprise Mortgage Investments, LLC': 'Enterprise',
    'AmeriSphere Multifamily Finance, L.L.C.': 'Amerisphere',
    'Greystone Servicing Corporation Inc.': 'Greyco',
    'ACRE Capital LLC': 'ACRE',
    'Berkeley Point Capital LLC': 'Berkeley',
    'M & T Realty Capital Corporation': 'M&T',
    'Capital One Multifamily Finance, LLC': 'Capital One',
    'Arbor Commercial Funding LLC': 'Arbor',
    'Red Mortgage Capital, LLC': 'Orix',
    'Pillar Multifamily, LLC': 'Pillar',
    'HomeStreet Capital Corporation': 'HomeStreet',
    'Oak Grove Commercial Mortgage, LLC': 'Oak Grove',
    'CBRE Multifamily Capital, Inc.': 'CBRE',
    'Citibank, N.A.': 'Citi',
    'Dougherty Mortgage, LLC': 'Dougherty',
    'Grandbridge Real Estate Capital LLC': 'Grandbridge',
    'KeyBank National Association': 'Key',
    'JP Morgan Chase Bank, NA': 'JPM',
    'NorthMarq Capital Finance, L.L.C.': 'NorthMarq',
    'Regions Bank': 'Regions',
    'Jones Lang LaSalle Multifamily, LLC.': 'JLL',
    'Hunt Mortgage Capital, LLC': 'Hunt',
    'Bellwether Enterprise Mortgage Investments, LLC': 'Bellwether',
    'Arbor Commercial Funding I, LLC': 'Arbor',
    'Barings Multifamily Capital LLC': 'Barings',
    'SunTrust Bank': 'Suntrust',
    'JPMorgan Chase Bank, NA': 'JPM',
    'PNC Bank, N.A.': 'PNC',
    'Berkadia Commercial Mortgage, LLC': 'Berkadia',
    'Arbor Commercial Funding, LLC': 'Arbor',
    'Red Mortgage Capital, Inc.': 'Orix',
    'KEYBANK NATIONAL ASSOCIATION': 'Key',
    'M&T Realty Capital Corporation': 'M&T',
    'Greystone Servicing Corporation': 'Greyco',
    'Berkeley Point Capital, LLC': 'Berkeley',
    'Jones Lang LaSalle Multifamily, LLC': 'JLL',
    'NorthMarq Capital Finance, LLC': 'NorthMarq',
    'Homestreet Capital Corporation': 'HomeStreet',
    'JPMorgan Chase Bank, N.A.': 'JPM',
    'Grandbridge Real Estate Capital, LLC': 'Grandbridge',
    'Capital One, National Association': 'Capital One',
    'Orix Real Estate Capital, LLC': 'Orix',
    'Greystone Servicing Company, LLC': 'Greyco',
    'Newmark Knight Frank': 'NKF',
    'Truist Bank': 'Truist',
    'COMMUNITY PRESERVATION CORPORATION': 'CPC',
    'BANK OF AMERICA, N.A.': 'BOA',
    'Prudential Multifamily Mortgage Inc.': 'Prudential',
    'PNC BANK, NATIONAL ASSOCIATION': 'PNC',
    'Alliant Capital LLC': 'Alliant',
    'Wells Fargo Bank N.A.': 'Wells',
    'Keycorp Real Estate Capital Markets, Inc': 'Key',
    'JPMORGAN CHASE BANK, NA': 'JPM',
    'GREYSTONE SERVICING CORPORATION INC.': 'Greyco',
    'ENTERPRISE MORTGAGE INVESTMENTS INC.': 'Enterprise',
    'HSBC Bank USA, NA': 'HSBC',
    'AMERISPHERE MULTIFAMILY FINANCE, L.L.C.': 'Amerisphere',
    'WELLS FARGO BANK N.A.': 'Wells',
    'MMA MORTGAGE INVESTMENT CORPORATION': 'MMA',
    'ARBOR COMMERCIAL FUNDING LLC': 'Arbor',
    'PRUDENTIAL MULTIFAMILY MORTGAGE INC.': 'Prudential',
    'PNC Multifamily Mortgage LLC': 'PNC',
    'ARCS COMMERCIAL MORTGAGE COMPANY, L.P.': 'PNC',
    'Deutsche Bank Berkshire Mortgage, Inc.': 'Deutsche',
    'WALKER & DUNLOP, LLC': 'Walker',
    'DEUTSCHE BANK BERKSHIRE MORTGAGE, INC.': 'Deutsche',
    'PNC ARCS LLC': 'PNC',
    'Enterprise Mortgage Investments Inc.': 'Enterprise',
    'Wachovia Multifamily Capital, Inc.': 'Wachovia',
    'MMA Mortgage Investment Corporation': 'MMA',
    'Capmark Finance Inc.': 'Capmark',
    'Walker & Dunlop LLC': 'Walker'
}
fnma['loan_seller'] = fnma['loan_seller'].replace(lender_rename_obj)

## New Fields

### Market Type

In [8]:
def find_market_type(row):
    
    state  = row['property_state']
    county = row['property_county']
    
    if state is None or county is None:
        return None
    
    try:
        state  = state.upper()
        county = county.lower()

        market = counties.loc[(counties['State']==state) & \
         ((counties['Google County'].str.contains(county, case=False)) | \
          (counties['HUD County'].str.contains(county, case=False))),
         'FNMA Market Tiers'
        ].values

        result = None
        if len(market) > 0:
            result = market[0]
    except:
        result = None
    
    return result
    
fnma['market_type'] = fnma.apply(find_market_type, axis=1)

fnma['market_type'] = fnma['market_type'].apply(lambda x: 'Pre-Review' if x=='Pre-ReviewNoDelegation' else x)

/Users/zr/miniconda3/envs/tc/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/Users/zr/miniconda3/envs/tc/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


### G&S

In [9]:
# There are three candidate note rate columns - 
# order of prioritization: loan_paying_note_rate_%_issuance, loan_original_note_rate_%, loan_paying_note_rate_%_current
fnma['note'] = np.where(~pd.isnull(fnma['loan_paying_note_rate_%_issuance']),
                            fnma['loan_paying_note_rate_%_issuance'],
                            np.where(~pd.isnull(fnma['loan_original_note_rate_%']),
                                 fnma['loan_original_note_rate_%'],
                                 fnma['loan_paying_note_rate_%_current']
                            )
                        )

# There are 2 candidate pass-thru rate (ptr) columns
# order of prioritization: loan_paying_ptr_%_issuance, loan_paying_ptr_%_current
fnma['ptr'] = np.where(~pd.isnull(fnma['loan_paying_ptr_%_issuance']),
                       fnma['loan_paying_ptr_%_issuance'],
                       fnma['loan_paying_ptr_%_current'],
                      )

# consolidate loan start date 
fnma['security_issue_date']     = pd.to_datetime(fnma['security_issue_date'])
fnma['loan_first_payment_date'] = pd.to_datetime(fnma['loan_first_payment_date'])
fnma['dt'] = fnma[['security_issue_date', 'loan_first_payment_date']].min(axis=1)

# drop records without a note rate, PTR or start date
fnma = fnma.dropna(subset=['note', 'ptr', 'dt'])

# Calculate G&S fee as spread between note rate and the pass-thru rate
fnma['gs'] = fnma['note'] - fnma['ptr']

---


In [10]:
fnma.loc[(fnma['loan_lien_position']!='First') & (fnma['dt']>'2019-5-1')]

,security_transaction_id,security_cusip,security_product_type,security_green,security_interest_type,security_upb_issuance,security_issue_date,loan_number,loan_interest_type,loan_amortization_type,...,property_3rd_preceding_year_egi,property_3rd_preceding_year_operating_expenses,property_3rd_preceding_year_ncf,property_3rd_preceding_year_physical_occupancy_%,property_type,market_type,note,ptr,dt,gs
18640,BL2081,3140HTJ34,DUS,N,Fixed,"368,000.00",2019-06-01,1717479266,Fixed,Amortizing/Balloon,...,NaN,NaN,NaN,NaN,Manufactured Housing,Pre-Review,5.680,3.660,2019-06-01,2.02
18715,BL2163,3140HTMM8,DUS,N,Fixed,"2,743,000.00",2019-07-01,1717479795,Fixed,Amortizing/Balloon,...,NaN,NaN,NaN,NaN,Manufactured Housing,Strong,5.140,2.890,2019-07-01,2.25
18871,BL2315,3140HTSD2,DUS,N,Fixed,"1,078,000.00",2019-07-01,1717479606,Fixed,Interest Only/Amortizing/Balloon,...,NaN,NaN,NaN,NaN,Manufactured Housing,Nationwide,5.330,3.590,2019-07-01,1.74
18991,BL2438,3140HTV89,DUS,N,Fixed,"970,000.00",2019-06-01,1717479355,Fixed,Amortizing/Balloon,...,NaN,NaN,NaN,NaN,Multifamily,Nationwide,5.780,4.040,2019-06-01,1.74
19051,BL2497,3140HTX38,DUS,N,Fixed,"2,910,000.00",2019-06-01,1717479357,Fixed,Interest Only/Amortizing/Balloon,...,NaN,NaN,NaN,NaN,Multifamily,Nationwide,5.460,3.260,2019-06-01,2.20
19103,BL2553,3140HTZT9,DUS,Y,Fixed,"5,715,000.00",2019-07-01,1717479823,Fixed,Amortizing/Balloon,...,NaN,NaN,NaN,NaN,Multifamily,Eligible,5.100,3.070,2019-07-01,2.03
19133,BL2588,3140HT2W8,DUS,N,Fixed,"17,680,000.00",2019-07-01,1717479828,Fixed,Interest Only/Balloon,...,NaN,NaN,NaN,NaN,Manufactured Housing,Nationwide,4.950,2.970,2019-07-01,1.98
19163,BL2611,3140HT3V9,DUS,N,Fixed,"827,000.00",2019-06-01,1717479372,Fixed,Amortizing/Balloon,...,NaN,NaN,NaN,NaN,Manufactured Housing,Eligible,5.710,3.630,2019-06-01,2.08
19174,BL2622,3140HT4G1,DUS,N,Fixed,"3,859,000.00",2019-06-01,1717479453,Fixed,Amortizing/Balloon,...,NaN,NaN,NaN,NaN,Multifamily,Pre-Review,5.280,3.080,2019-06-01,2.20
19175,BL2623,3140HT4H9,DUS,N,Fixed,"867,000.00",2019-06-01,1717479323,Fixed,Interest Only/Amortizing/Balloon,...,NaN,NaN,NaN,NaN,Manufactured Housing,Pre-Review,5.460,3.630,2019-06-01,1.83


In [11]:
# Record counts by year
fnma['dt'].apply(lambda dt: dt.year) \
    .value_counts()

2019    3792
2018    3247
2017    2926
2016    2293
2015    1848
2020    1668
2012    1579
2013    1535
2014    1325
2011     826
2010     443
2007     163
2006     154
2008     129
2005     117
2009      96
2003      44
2004      43
2000      34
2002      14
2001       7
1997       5
1998       5
1999       4
Name: dt, dtype: int64

In [12]:
fnma[['note', 'ptr', 'gs']].describe()

,note,ptr,gs
count,22297.000000,22297.000000,22297.000000
mean,4.485565,3.192103,1.293461
std,0.790450,0.815534,0.419008
min,1.940000,0.870000,0.120000
25%,3.970000,2.660000,1.010000
50%,4.400000,3.080000,1.290000
75%,4.860000,3.560000,1.520000
max,9.650000,8.400000,5.010000


# Visualize

## Setup Graph

In [13]:
# Sort by Loan Start Date
fnma = fnma.sort_values(by='dt')

lender_list = list(fnma.loc[~pd.isnull(fnma['loan_seller']),'loan_seller'].unique())

no_color    = 'rgba(255,255,255,0)' # used for upper/lower bound plotlines

In [14]:
# Graph functions
def reset_fig(fig):
    fig.data   = []
    fig.layout = {}
    return fig

def init_fig(title):
    fig = go.FigureWidget()

    fig.update_layout(
        template='plotly_white', 
        title=title,
        width=1200,
        height=800,
        xaxis={
            'rangeslider': {
                'visible': True
            },
            'type': "date"
        }
    )

    return fig

def add_line(fig, x, y, y_upper=None, y_lower=None, color=None, name=None):
    color = color or palette[list(palette.keys())[random.randint(0, len(palette))]] # select random color if none provided
    
    if y_upper is not None:
        # Upper Bound
        fig.add_trace(go.Scatter(
            x=x, y=y_upper, 
            name        = name + '_upper',
            line_color  = no_color,
            showlegend  = False,
            legendgroup = name
        ))
    
    # Mean
    fig.add_trace(go.Scatter(
        x=x, y=y, 
        name        = name,
        fill        = 'tonexty',
        line_color  = color['primary'],
        fillcolor   = color['fill'],
        legendgroup = name
    ))

    if y_lower is not None:
        # Lower Bound
        fig.add_trace(go.Scatter(
            x=x, y=y_lower, 
            name        = name + '_lower',
            fill        = 'tonexty', 
            line_color  = no_color,
            fillcolor   = color['fill'],
            showlegend  = False,
            legendgroup = name
        ))

def get_line_by_name(fig, name, get_bounds=True):
    line = list(filter(lambda line: line['name'] == name, fig.data))[0]
    if get_bounds:
        line_upper = list(filter(lambda line: line['name'] == name+'_upper', fig.data))[0]
        line_lower = list(filter(lambda line: line['name'] == name+'_lower', fig.data))[0]
        return [line, line_upper, line_lower]
    else:
        return [line]
        
def update_chart(change):
    '''
    Would probably be more efficient to seperate this update into multiple depending on which widget made the change.
    However, the performance isn't bad and, well, ¯\_(ツ)_/¯
    '''

    data = fnma.copy()
    
    # The below looks at each widget value and constructs a single query string
    query = ''

    # ---- Input Ranges ----
    date_min, date_max = date_range.value
    query += f"(dt>='{date_min}' & dt<='{date_max}')"
    
    units_min, units_max = units_range.value
    query += f" & (property_total_units>={units_min} & property_total_units<={units_max})"
    
    loan_amt_min, loan_amt_max = loan_amt_range.value
    query += f" & (loan_original_upb >= {loan_amt_min} & loan_original_upb <= {loan_amt_max})"
    
    term_min, term_max = loan_term_range.value
    query += f" & (loan_original_term>={term_min} & loan_original_term<={term_max})"

    # ---- Input Dropdowns ----
    if affordable.value == 'Any':
        query += f" & (property_affordable_housing_type != 'None')"
    elif affordable.value != 'All':
        query += f" & (property_affordable_housing_type == '{affordable.value}')"
    if green.value != 'All':
        query += f" & (security_green == '{green.value}')"
    if lien_position.value != 'All':
        query += f" & (loan_lien_position == '{lien_position.value}')"
    if tier.value != 'All':
        query += f" & (loan_tier == '{tier.value}')"
    if prepay_type.value != 'All':
        query += f" & (loan_prepayment_type == '{prepay_type.value}')"
    if property_type.value != 'All':
        query += f" & (property_type == '{property_type.value}')"
    if market_type.value != 'All':
        query += f" & (market_type == '{market_type.value}')"

    data         = data.query(query)
    data_grouped = data.groupby('dt').agg({'gs': ['min', 'max', 'mean']})['gs']
    
    with fig.batch_update():
        fig.layout.title.text = f"G&S Fees - {len(data)} records"
        
        # Update the 'All' line first
        x       = list(data_grouped.index)
        y       = data_grouped['mean']
        y_upper = data_grouped['max']
        y_lower = data_grouped['min']
        
        line_main, line_upper, line_lower = get_line_by_name(fig, 'All', get_bounds=True)

        line_main.x = x
        line_main.y = y
        
        line_upper.x = x
        line_upper.y = y_upper
        
        line_lower.x = x
        line_lower.y = y_lower

        
        # Update lines for each lender
        for lender in lender_list:
            
            lender_data    = data[data['loan_seller']==lender].sort_values(by='dt')
            lender_grouped = lender_data.groupby('dt').agg({'gs': ['min', 'max', 'mean']})['gs']

            x       = list(lender_grouped.index)
            y       = lender_grouped['mean']
            y_upper = lender_grouped['max']
            y_lower = lender_grouped['min']
            
            line_main, line_upper, line_lower = get_line_by_name(fig, lender, get_bounds=True)
            
            line_main.x = x
            line_main.y = y

            line_upper.x = x
            line_upper.y = y_upper

            line_lower.x = x
            line_lower.y = y_lower


In [15]:
# Graph Widgets
dates      = pd.date_range(fnma['dt'].min(), fnma['dt'].max(), freq='M')
options    = [(date.strftime(' %d %b %Y '), date) for date in dates]
date_range = widgets.SelectionRangeSlider(
    continuous_update=False,
    options=options,
    index=(0, len(options)-1),
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)

units_range = widgets.SelectionRangeSlider(
    continuous_update=False,
    options=list(fnma['property_total_units'].sort_values().unique()),
    index=(0, len(fnma['property_total_units'].unique())-1),
    description='Units',
    orientation='horizontal',
    layout={'width': '500px'}
)

loan_amt_range = widgets.SelectionRangeSlider(
    continuous_update=False,
    options=list(fnma['loan_original_upb'].sort_values().unique()),
    index=(0, len(fnma['loan_original_upb'].unique())-1),
    description='Loan Amount',
    orientation='horizontal',
    layout={'width': '500px'}
)

loan_term_range = widgets.SelectionRangeSlider(
    continuous_update=False,
    options=list(fnma['loan_original_term'].sort_values().unique()),
    index=(0, len(fnma['loan_original_term'].unique())-1),
    description='Loan Term',
    orientation='horizontal',
    layout={'width': '500px'}
)

affordable = widgets.Dropdown(
    options=list(fnma['property_affordable_housing_type'].unique())+['All', 'Any'],
    value='All',
    description='Affordable: ',
)

green = widgets.Dropdown(
    options=list(fnma['security_green'].unique())+['All'],
    value='All',
    description='Green: ',
)

lien_position = widgets.Dropdown(
    options=list(fnma['loan_lien_position'].unique())+['All'],
    value='All',
    description='Lien Pos: ',
)

tier = widgets.Dropdown(
    options=list(fnma['loan_tier'].unique())+['All'],
    value='All',
    description='Tier: ',
)

prepay_type = widgets.Dropdown(
    options=list(fnma['loan_prepayment_type'].unique())+['All'],
    value='All',
    description='Prepay Type: ',
)

property_type = widgets.Dropdown(
    options=list(fnma['property_type'].unique())+['All'],
    value='All',
    description='Property Type: ',
)

market_type = widgets.Dropdown(
    options=list(fnma['market_type'].unique())+['All'],
    value='All',
    description='Market Type: ',
)

# Hook listeners to widgets
affordable.observe(update_chart)
green.observe(update_chart)
lien_position.observe(update_chart)
tier.observe(update_chart)
date_range.observe(update_chart)
prepay_type.observe(update_chart)
property_type.observe(update_chart)
units_range.observe(update_chart)
loan_amt_range.observe(update_chart)
loan_term_range.observe(update_chart)
market_type.observe(update_chart)

In [19]:
cols = [
    'dt',
    'gs',
    'loan_original_term',
    'loan_original_upb',
    'property_total_units',
    'loan_seller',
    'market_type property_type',
    'loan_prepayment_type',
    'loan_tier',
    'loan_lien_position',
    'security_green',
    'property_affordable_housing_type'
]
fnma.head()

,security_transaction_id,security_cusip,security_product_type,security_green,security_interest_type,security_upb_issuance,security_issue_date,loan_number,loan_interest_type,loan_amortization_type,...,property_3rd_preceding_year_egi,property_3rd_preceding_year_operating_expenses,property_3rd_preceding_year_ncf,property_3rd_preceding_year_physical_occupancy_%,property_type,market_type,note,ptr,dt,gs
385,375129,31377EVW6,DUS,N,Fixed,"1,143,000.00",1997-05-01,1665701138,Fixed,Fully Amortizing,...,"746,122.00","520,248.00","190,575.00",94.0,Multifamily,None,8.75,7.900,1997-05-01,0.850
386,375173,31377EXA2,DUS,N,Fixed,"2,400,000.00",1997-05-01,1665707841,Fixed,Fully Amortizing,...,"864,089.00","483,412.00","352,957.00",98.0,Multifamily,None,8.62,7.545,1997-05-01,1.075
387,375283,31377E2Q1,DUS,N,Fixed,"1,000,000.00",1997-07-01,1665868326,Fixed,Fully Amortizing,...,"1,626,749.65","1,466,314.76","132,434.89",100.0,Cooperative,None,7.90,7.250,1997-07-01,0.650
388,375487,31377FCY0,DUS,N,Fixed,"940,000.00",1997-10-01,1666355853,Fixed,Fully Amortizing,...,"416,258.00","222,092.00","180,075.00",100.0,Multifamily,None,8.43,7.280,1997-10-01,1.150
389,375531,31377FEC6,DUS,N,Fixed,"865,000.00",1997-12-01,1666519243,Fixed,Fully Amortizing,...,"425,990.00","283,511.51","136,478.49",100.0,Cooperative,None,7.39,7.040,1997-12-01,0.350


In [16]:
# Create Fig
fig = init_fig(title='G&S Fees')

# Seed fig (hahaha) with some initial data

data = fnma.groupby('dt').agg({'gs': ['min', 'max', 'mean']})['gs']

x       = list(data.index)
y       = data['mean']
y_upper = data['max']
y_lower = data['min']

add_line(fig, x, y, y_upper=y_upper, y_lower=y_lower, name='All')

# Add plot lines for each lender
for i, lender in enumerate(lender_list):
    lender_data    = fnma[fnma['loan_seller']==lender].sort_values(by='dt')
    lender_grouped = lender_data.groupby('dt').agg({'gs': ['min', 'max', 'mean']})['gs']

    x       = list(lender_grouped.index)
    y       = lender_grouped['mean']
    y_upper = lender_grouped['max']
    y_lower = lender_grouped['min']
    
    # color:
    if i >= len(palette.keys()):
        # Start from beginning of color palette
        color_key = list(palette.keys())[i - len(palette.keys())]
    else:
        color_key = list(palette.keys())[i]
    color     = palette[color_key]
    
    add_line(fig, x, y, y_upper=y_upper, y_lower=y_lower, name=lender, color=color)

## Graph

In [17]:
# Create widget grid with widgets and fig
widgets.VBox([
    widgets.HBox([date_range]), 
    fig, 
    widgets.HBox([affordable, green, lien_position, tier]), 
    widgets.HBox([prepay_type, property_type, market_type]),
    widgets.HBox([loan_amt_range, units_range, loan_term_range]),
])

Findings:
Green Rewards - across the board CBRE seems to have an upper hand versus Hunt

Traditional 10 yr, t2 ym mkt rate - CBRE & PNC take bottom with Hunt & HomeStreet on top

In [23]:
fig.write_html("chart.html")